In [1]:
import pandas as pd
import numpy as np
import xgboost

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns

In [92]:
train_df = pd.read_csv('datasets/loan_added_features.csv')

In [93]:
train_df.head()

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,LoanAmount_log,TotalIncome,TotalIncome_log,EMI,BalanceIncome
0,0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1,4.852030,5849.0,8.674026,0.355556,5493.444444
1,1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,4.852030,6091.0,8.714568,0.355556,5735.444444
2,2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,4.189655,3000.0,8.006368,0.183333,2816.666667
3,3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,4.787492,4941.0,8.505323,0.333333,4607.666667
4,4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,4.948760,6000.0,8.699515,0.391667,5608.333333


In [94]:
#train_df['BalanceIncome_log'] = np.log(train_df['BalanceIncome'])
train_df['EMI_log'] = np.log(train_df['EMI'])
train_df.head()

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,LoanAmount_log,TotalIncome,TotalIncome_log,EMI,BalanceIncome,EMI_log
0,0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,1,4.852030,5849.0,8.674026,0.355556,5493.444444,-1.034074
1,1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,4.852030,6091.0,8.714568,0.355556,5735.444444,-1.034074
2,2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,4.189655,3000.0,8.006368,0.183333,2816.666667,-1.696449
3,3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,4.787492,4941.0,8.505323,0.333333,4607.666667,-1.098612
4,4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,4.948760,6000.0,8.699515,0.391667,5608.333333,-0.937344


In [95]:
## Removing some columns which will not make a difference based on initial analysis.

train_df.drop(labels=['ApplicantIncome', 'CoapplicantIncome', 'Unnamed: 0', 'EMI', 'TotalIncome', 'Loan_ID', 'LoanAmount' ], axis=1, inplace=True)

In [96]:
train_df.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'LoanAmount_log', 'TotalIncome_log',
       'BalanceIncome', 'EMI_log'], dtype=object)

In [97]:
train_df['Gender'].replace({'Male': 0,
                            'Female': 1},
                 inplace=True)

In [98]:
train_df['Education'].replace({'Not Graduate': 0,
                               'Graduate': 1},
                              inplace=True)
train_df['Married'].replace({'No': 0,
                             'Yes': 1},
                             inplace=True)
train_df['Self_Employed'].replace({'No': 0,
                                   'Yes': 1},
                                   inplace=True)

In [99]:
print(train_df.Gender.unique())
print(train_df.Education.unique())
print(train_df.Dependents.unique())
print(train_df.Married.unique())
print(train_df.Self_Employed.unique())
print(train_df.Credit_History.unique())
print(train_df.Property_Area.unique())

[0 1]
[1 0]
[0. 1. 2. 3.]
[0 1]
[0 1]
[1. 0.]
['Urban' 'Rural' 'Semiurban']


In [64]:
print(train_df.Gender.unique())
print(train_df.Education.unique())
print(train_df.Dependents.unique())
print(train_df.Married.unique())
print(train_df.Self_Employed.unique())
print(train_df.Credit_History.unique())

[0 1]
[1 0]
[0. 1. 2. 3.]
[0 1]
[0 1]
[1. 0.]


In [108]:
train_df_temp = pd.get_dummies(train_df, columns=['Property_Area'])

In [109]:
train_df_temp.head()

,Gender,Married,Dependents,Education,Self_Employed,Loan_Amount_Term,Credit_History,Loan_Status,LoanAmount_log,TotalIncome_log,BalanceIncome,EMI_log,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0,0,0.0,1,0,360.0,1.0,1,4.852030,8.674026,5493.444444,-1.034074,0,0,1
1,0,1,1.0,1,0,360.0,1.0,0,4.852030,8.714568,5735.444444,-1.034074,1,0,0
2,0,1,0.0,1,1,360.0,1.0,1,4.189655,8.006368,2816.666667,-1.696449,0,0,1
3,0,1,0.0,0,0,360.0,1.0,1,4.787492,8.505323,4607.666667,-1.098612,0,0,1
4,0,0,0.0,1,0,360.0,1.0,1,4.948760,8.699515,5608.333333,-0.937344,0,0,1


In [ ]:
#Loan_Amount, Loan_Amount_term, 